# Data Pre-Processing

Is your data too messy to be utilized in 02? Look no further! This notebook walks through the data pre-processing methodology for our datasets, particularly BDD100K. We also include some helpful tips to make your data more compatible with these notebooks.

## BDD100K


In [ ]:
import networkx as nx
import osmnx as ox 
import time
from shapely.geometry import Polygon
import os, io, sys
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from algorithms import mm_utils

%matplotlib inline
#ox.__version__

/home/gjgress/.local/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Importing your Data

If you are seriously testing your algorithm against data, chances are your dataset is huge. Blindly trying to import it into a Pandas (Geo)DataFrame is going to cause some issues, because it will attempt to load it all into memory (which is likely impossible).

In our case, we use Dask to handle this.

(In general, if you are exclusively using Pandas, Modin might be easier, as it is drop-in compatible. But Dask can handle GeoDataFrames (unlike Modin), so we will use that here) 

In [ ]:
# Fast JSON library
import ujson as json

## Note for future self: If you need to muck around with JSON formatting, jq might be the way to go
## Transforming to/from line-delimited, for example, is far simpler

# Import the Dask libraries we need
import dask.bag as db

In [ ]:
# If you try to load the BDD100K files directly into Dask, you'll run into some issues.
# Dask Bags assume every line in a json file is a distinct json object.
# So because BDD100K uses pretty json formatting, it will not load properly.
# So we have to first remove all the newline characters in all of the files.

# Note that the database size is huge, so make sure you have adequate disk space

path = 'BDD100K/train/'

for filename in os.listdir(path):
    if ('json' in filename and not 'processed' in filename):
        filepath = os.path.join(path, filename)
        f = open(filepath, 'r')
        f = f.read().replace('\n', '')
        if 'gps' in f.read(): # Some of the files don't have GPS data.
# These are unusuable to us, so we don't want them
# In BDD100K, there's about 20000 files like this... quite unfortunate
            if os.path.getsize(os.path.join(path,filename)) == 0: 
    # Fun fact: the BDD100K info dataset has a corrupted (empty) JSON file
    # This caused an enormous headache on my end while debugging
    # So now we will only process it if it's non-empty.
    # Otherwise, we skip the processing part, but still delete the file
    # (A more robust method would be to `try: json.loads`, but this would greatly increase processing time)
                with open(path + 'processed-' + filename, 'w') as fp:
                    print(f, file=fp)
        # These files take up a lot of space on my harddrive, so I will remove them here.
        # You may wish not to do this
        os.remove(filepath)


We went ahead and removed newline characters from our files, and threw away incompatible JSON files. Now we load the JSON files into our Dask bag, and initialize our reformatting function.

This function is customized to the BDD100K dataset to pull the necessary info out and reformat it into GeoJSON. You will have to write your own function customized for whatever dataset you choose to use.

In [ ]:
# Now we load all the JSON files

dfbag = db.read_text('BDD100K/train/processed-*.json').map(json.loads)

# This is a helper function for reformatting

def bdd_reformat(jsonf):        
    listf = []
    if jsonf.get('gps') != None:
        for item in jsonf['gps']:
            listf.append({"type": "Feature",
          "geometry": {
            "type": "Point",
            "coordinates": [item["longitude"], item["latitude"]]
          },
          "properties": {
            "timestamp": item["timestamp"],
            "altitude": item["altitude"],
            "speed": item["speed"],
            "vertical accuracy": item["vertical accuracy"],
            "horizontal accuracy": item["horizontal accuracy"]
          }})
        geojsonf = {"type": "FeatureCollection", "features": listf}
        return geojsonf
    else:
        return jsonf

ValueError: ('No files found', 'BDD100K/train/processed-*.json')

While our data is now in GeoJSON, it is still stored as a Python dictionary. So we will export the files so we don't have to repeat this process later.

In [ ]:
path = 'BDD100K/train/'
files = 'postprocessed-*.geojson'

# create a text trap and redirect stdout
text_trap = io.StringIO()
sys.stdout = text_trap

### TODO: Consider outputting to better file format, e.g. Avro, Parquet

dfbag = dfbag.map(bdd_reformat)
dfbag.map(json.dumps).to_textfiles(path + files)

# now restore stdout function
sys.stdout = sys.__stdout__

# I don't have the harddrive space to store both the intermediate and final files
# So I delete the old ones here
# You may prefer to not do this
for filename in os.listdir(path):
    if (not 'postprocessed' in filename):
        filepath = os.path.join(path, filename)
        os.remove(filepath)

# Note-- now may be a good time to zip and compress the processed files, in case something happens

NameError: name 'dfbag' is not defined

Now our data has been post-processed to a format that is compatible with map matching algorithms. The simplest way to utilize the data is to load it all into a Dask Bag, and `take(n,npartitions=n)` as needed (alternatively, you can load each GeoJSON as a partition in a Dask GeoDataFrame-- but this has complications). However, if you wish to do more in-depth data analysis on the dataset, thousands of JSON files aren't exactly optimal. We could try to apply functions on Dask Bags, but the simpler solution is to store the files into a SQLite database. Then we can access the database as needed and access filtered data quickly. 

Note: if you have no interest in utilizing the GeoJSON structure, you should create a database from the unprocessed files

In [ ]:
# In our case, it makes more sense to store it into a SQLite database, but MySQL, MariaDB, or other formats work perfectly well.

# Fortunately, there are a lot of tools to convert GeoJSON to a spatially informed database
# So instead of trying to do it ourselves, we will use an external tool to do the heavy lifting
# Aren't you glad we processed the data into a more standard format?

# Run this only once
#! pip install geojson-to-sqlite
#! sudo pamac install spatialite-gui # Optional, but improves our database
#dfbag = db.read_text('BDD100K/train/postprocessed-*.json').map(json.loads)

In [ ]:
path = 'BDD100K/train/'

# I am not adept at SQL, so I don't know the best way to store these files in an SQL table
# As a result, I will use an external program which converts GPS tracks to an SQL database
# However, it requires our files to be GPX, so let's do that first...

! bash geojson_to_gpx.sh

In [ ]:
! gpx2spatialite create_db BDD100K/postprocessed_BDD100K.d
! gpx2spatialite import -d BDD100K/postprocessed_BDD100K.db -u Gabe BDD100K/train/*.gpx

Let's run a query to make sure it works.

In [ ]:
### WIP


#import sqlite3
#conn = sqlite3.connect('BDD100K/postprocessed-BDD100K.db')
#conn.enable_load_extension(True)

# Now we load spatialite
#conn.execute('SELECT load_extension("mod_spatialite")')
#conn.execute('SELECT InitSpatialMetaData(1);')

# libspatialite
#conn.execute('SELECT load_extension("libspatialite")')
#conn.execute('SELECT InitSpatialMetaData();')

#cur = conn.cursor()
#cur.execute('SELECT ')

#conn.commit()
#conn.close()
#del conn

All done, right? Not quite. For example: is your data fused?

In [ ]:
## Display data and see if fused

# Note: If you chose to store all your data in one SQL Table,
# Dask DataFrames can import from that.
gdfbag = dfbag.map(gpd.GeoDataFrame.from_features)

In our case, our data is already fused. But often you will have several datasets with asynchronous data that you will have to fuse first. We implemented a barebones method in mm_utils to handle this; here is an example of how to apply it.

Note that your data needs to be a (Geo)DataFrame or GeoJSON. Also, the first column of all the datasets needs to be the time, and must all share the same time formatting. If you aren't sure your time format will work, we recommmend converting it all to Unix time (most languages have a built-in method to do this)

In [ ]:
# We create simulated asynchronous data

# Get columns of data
origdf = gdfbag.take(1)[0]

/home/gjgress/.local/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
speed = origdf[['timestamp','speed']]

# Create rng to create noisy data
rng = np.random.default_rng()
unfused_spd = pd.DataFrame([])
for i in speed.index[:-1]:
    # Generate speed values based on original, at random timestamps
    row1 = pd.DataFrame([[speed.iloc[i][0] + rng.random()*1000,(speed.iloc[i][1] + speed.iloc[i+1][1])/2 + np.random.normal(0, (np.abs(speed.iloc[i][1] - speed.iloc[i+1][1]))/4)]], columns = ['timestamp','speed'], index = [i])
    unfused_spd = pd.concat([unfused_spd, row])
# Reset indices
unfused_spd = unfused_spd.sort_index().reset_index(drop=True)
unfused_spd

,timestamp,speed
0,1.503834e+12,4.695797
1,1.503834e+12,4.479840
2,1.503834e+12,4.365951
3,1.503834e+12,4.174768
4,1.503834e+12,3.970857
5,1.503834e+12,3.870786
6,1.503834e+12,3.882143
7,1.503834e+12,4.311895
8,1.503834e+12,4.999623
9,1.503834e+12,5.771902


Now we wish to fuse this DataFrame with our original dataframe (excluding the original speed column)

In [ ]:
df1 = origdf[['timestamp', 'altitude', 'vertical accuracy', 'horizontal accuracy', 'geometry']]

df_prox = mm_utils.fuse(df1,unfused_spd,'timestamp','nearest neighbor')
df_avg = mm_utils.fuse(df1,unfused_spd,'timestamp','average')

Now let's see what the speed columns look like side-by-side

In [ ]:
# This cell sets up styling to facilitate comparison
from IPython.display import display_html

df_sty = origdf[['speed']].style.set_table_attributes("style='display:inline'").set_caption('original df')
df_prox_sty = df_prox[['speed']].style.set_table_attributes("style='display:inline'").set_caption('proximity fuse')
df_avg_sty = df_avg[['speed']].style.set_table_attributes("style='display:inline'").set_caption('average fuse')

In [ ]:
space = "\xa0" * 10
display_html(df_sty._repr_html_() + space
             + df_prox_sty._repr_html_() + space
             + df_avg_sty._repr_html_(), raw=True)


,speed
0,5.670000
1,4.610000
2,4.390000
3,4.350000
4,4.100000
5,3.890000
6,3.860000
7,3.890000
8,4.500000
9,5.610000


Because these are undersampled data points with heavy noise, the averaging method performs rather poorly. If sample size is far larger than the main dataset, average performs a lot better-- but then nearest neighbor is weak to high variance (as it only considers the closest point, which may be an outlier.

## Large-Scale Dataset

First, we import the data and write it as GEOJson

In [ ]:
# Processing the coordinate track first

import csv
from shapely.geometry import Point
from shapely.geometry import LineString
path = 'map-matching-dataset/'


In [ ]:

for filename in os.listdir(path):
    if '.track' in filename:
        gdf = gpd.GeoDataFrame([])
        
        filepath = os.path.join(path, filename)
        with open(filepath,newline='') as csvf:
            data = csv.reader(csvf,delimiter = '\t')
            # To be honest this is a lazy approach, but we only have 100 tracks anyway, so its fine...
            for row in data:
                pt = Point(float(row[0]), float(row[1]))
                gdf = pd.concat([gdf, gpd.GeoDataFrame([{'timestamp':row[2], 'geometry':pt}])])
        gdf.to_file(filepath[:-6] + 'track' + '.geojson', driver = 'GeoJSON')

In [ ]:
# Now we construct our network

def string_to_coords(string):
    return tuple(map(float,string.split('\t')))

def string_to_ints(string):
    return tuple(map(int,string.split('\t')))


def helper_fn1(df,ls):
    return df.iloc[ls]

def helper_fn2(ls):
    return LineString(list(ls.loc[:,'geometry']))

for filename in os.listdir(path):
    if '.nodes' in filename:
        filepath = os.path.join(path, filename)
        with open(filepath,newline='') as f:
            fdata = f.read()
            flist = fdata.split('\n')
            flist = list(filter(None, flist))
            flist = map(Point,list(map(string_to_coords,flist)))
            ngdf = gpd.GeoDataFrame(flist, columns=['geometry'])
            # We need to do arcs while we do nodes, b.c. it is indexed by rows in .nodes
            with open(filepath[:-6] + '.arcs', newline='') as f2:
                arcdata = f2.read()
                arclist = arcdata.split('\n')
                arclist = list(filter(None,arclist))
                arclist = list(map(string_to_ints,arclist))
                arclistu = [] # We need to store the source and target nodes
                arclistv = []
                for i in range(len(arclist)):
                    arclistu.append(arclist[i][0])
                    arclistv.append(arclist[i][1])
                arclist = map(list, arclist)
                arcpts = list(map(lambda x: helper_fn1(ngdf,x), arclist))
                arcptsid = list(map(lambda x: helper_fn1(ngdf,x), arclist))
                arclist = list(map(helper_fn2, arcpts))
                agdf = gpd.GeoDataFrame({'geometry' : arclist, 'u' : arclistu, 'v' : arclistv})
        ngdf.to_file(filepath[:-6] + 'nodes' + '.geojson', driver = 'GeoJSON')
        agdf.to_file(filepath[:-6] + 'arcs' + '.geojson', driver = 'GeoJSON')

In [ ]:
# Ground Truth
# This is the RIGHT way to do this... much faster. When I get time, rewrite the previous code block
for filename in os.listdir(path):
    if 'arcs.geojson' in filename:
        filepath = os.path.join(path, filename)
        agdf = gpd.read_file(filepath)
        with open(filepath[:-12] + '.route','r') as f:
            data = f.read()
            dlist = data.split("\n")
            dlist = list(filter(None, dlist))
            dlist = list(map(int,dlist))
            rgdf = agdf.iloc[dlist]
        rgdf.to_file(filepath[:-12] + 'route' + '.geojson', driver = 'GeoJSON', index = True)


In [ ]:
for filename in os.listdir(path):
    if ('geojson' in filename):
        filepath = os.path.join(path, filename)
        with open(filepath, 'r+') as f:
            text = f.read().replace('\n', '')
            f.seek(0)
            f.write(text)
            f.truncate()

0

300442

300442

0

1199263

...

586345

586345

0

6677294

6677294

# Data Pre-Processing

Is your data too messy to be utilized in 02? Look no further! This notebook walks through the data pre-processing methodology for our datasets, particularly BDD100K. We also include some helpful tips to make your data more compatible with these notebooks.

## BDD100K


In [ ]:
import networkx as nx
import osmnx as ox 
import time
from shapely.geometry import Polygon
import os, io, sys
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from algorithms import mm_utils

%matplotlib inline
#ox.__version__

/home/gjgress/.local/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Importing your Data

If you are seriously testing your algorithm against data, chances are your dataset is huge. Blindly trying to import it into a Pandas (Geo)DataFrame is going to cause some issues, because it will attempt to load it all into memory (which is likely impossible).

In our case, we use Dask to handle this.

(In general, if you are exclusively using Pandas, Modin might be easier, as it is drop-in compatible. But Dask can handle GeoDataFrames (unlike Modin), so we will use that here) 

In [ ]:
# Fast JSON library
import ujson as json

## Note for future self: If you need to muck around with JSON formatting, jq might be the way to go
## Transforming to/from line-delimited, for example, is far simpler

# Import the Dask libraries we need
import dask.bag as db

In [ ]:
# If you try to load the BDD100K files directly into Dask, you'll run into some issues.
# Dask Bags assume every line in a json file is a distinct json object.
# So because BDD100K uses pretty json formatting, it will not load properly.
# So we have to first remove all the newline characters in all of the files.

# Note that the database size is huge, so make sure you have adequate disk space

path = 'BDD100K/train/'

for filename in os.listdir(path):
    if ('json' in filename and not 'processed' in filename):
        filepath = os.path.join(path, filename)
        f = open(filepath, 'r')
        f = f.read().replace('\n', '')
        if 'gps' in f.read(): # Some of the files don't have GPS data.
# These are unusuable to us, so we don't want them
# In BDD100K, there's about 20000 files like this... quite unfortunate
            if os.path.getsize(os.path.join(path,filename)) == 0: 
    # Fun fact: the BDD100K info dataset has a corrupted (empty) JSON file
    # This caused an enormous headache on my end while debugging
    # So now we will only process it if it's non-empty.
    # Otherwise, we skip the processing part, but still delete the file
    # (A more robust method would be to `try: json.loads`, but this would greatly increase processing time)
                with open(path + 'processed-' + filename, 'w') as fp:
                    print(f, file=fp)
        # These files take up a lot of space on my harddrive, so I will remove them here.
        # You may wish not to do this
        os.remove(filepath)


We went ahead and removed newline characters from our files, and threw away incompatible JSON files. Now we load the JSON files into our Dask bag, and initialize our reformatting function.

This function is customized to the BDD100K dataset to pull the necessary info out and reformat it into GeoJSON. You will have to write your own function customized for whatever dataset you choose to use.

In [ ]:
# Now we load all the JSON files

dfbag = db.read_text('BDD100K/train/processed-*.json').map(json.loads)

# This is a helper function for reformatting

def bdd_reformat(jsonf):        
    listf = []
    if jsonf.get('gps') != None:
        for item in jsonf['gps']:
            listf.append({"type": "Feature",
          "geometry": {
            "type": "Point",
            "coordinates": [item["longitude"], item["latitude"]]
          },
          "properties": {
            "timestamp": item["timestamp"],
            "altitude": item["altitude"],
            "speed": item["speed"],
            "vertical accuracy": item["vertical accuracy"],
            "horizontal accuracy": item["horizontal accuracy"]
          }})
        geojsonf = {"type": "FeatureCollection", "features": listf}
        return geojsonf
    else:
        return jsonf

ValueError: ('No files found', 'BDD100K/train/processed-*.json')

While our data is now in GeoJSON, it is still stored as a Python dictionary. So we will export the files so we don't have to repeat this process later.

In [ ]:
path = 'BDD100K/train/'
files = 'postprocessed-*.geojson'

# create a text trap and redirect stdout
text_trap = io.StringIO()
sys.stdout = text_trap

### TODO: Consider outputting to better file format, e.g. Avro, Parquet

dfbag = dfbag.map(bdd_reformat)
dfbag.map(json.dumps).to_textfiles(path + files)

# now restore stdout function
sys.stdout = sys.__stdout__

# I don't have the harddrive space to store both the intermediate and final files
# So I delete the old ones here
# You may prefer to not do this
for filename in os.listdir(path):
    if (not 'postprocessed' in filename):
        filepath = os.path.join(path, filename)
        os.remove(filepath)

# Note-- now may be a good time to zip and compress the processed files, in case something happens

NameError: name 'dfbag' is not defined

Now our data has been post-processed to a format that is compatible with map matching algorithms. The simplest way to utilize the data is to load it all into a Dask Bag, and `take(n,npartitions=n)` as needed (alternatively, you can load each GeoJSON as a partition in a Dask GeoDataFrame-- but this has complications). However, if you wish to do more in-depth data analysis on the dataset, thousands of JSON files aren't exactly optimal. We could try to apply functions on Dask Bags, but the simpler solution is to store the files into a SQLite database. Then we can access the database as needed and access filtered data quickly. 

Note: if you have no interest in utilizing the GeoJSON structure, you should create a database from the unprocessed files

In [ ]:
# In our case, it makes more sense to store it into a SQLite database, but MySQL, MariaDB, or other formats work perfectly well.

# Fortunately, there are a lot of tools to convert GeoJSON to a spatially informed database
# So instead of trying to do it ourselves, we will use an external tool to do the heavy lifting
# Aren't you glad we processed the data into a more standard format?

# Run this only once
#! pip install geojson-to-sqlite
#! sudo pamac install spatialite-gui # Optional, but improves our database
#dfbag = db.read_text('BDD100K/train/postprocessed-*.json').map(json.loads)

In [ ]:
path = 'BDD100K/train/'

# I am not adept at SQL, so I don't know the best way to store these files in an SQL table
# As a result, I will use an external program which converts GPS tracks to an SQL database
# However, it requires our files to be GPX, so let's do that first...

! bash geojson_to_gpx.sh

In [ ]:
! gpx2spatialite create_db BDD100K/postprocessed_BDD100K.d
! gpx2spatialite import -d BDD100K/postprocessed_BDD100K.db -u Gabe BDD100K/train/*.gpx

Let's run a query to make sure it works.

In [ ]:
### WIP


#import sqlite3
#conn = sqlite3.connect('BDD100K/postprocessed-BDD100K.db')
#conn.enable_load_extension(True)

# Now we load spatialite
#conn.execute('SELECT load_extension("mod_spatialite")')
#conn.execute('SELECT InitSpatialMetaData(1);')

# libspatialite
#conn.execute('SELECT load_extension("libspatialite")')
#conn.execute('SELECT InitSpatialMetaData();')

#cur = conn.cursor()
#cur.execute('SELECT ')

#conn.commit()
#conn.close()
#del conn

All done, right? Not quite. For example: is your data fused?

In [ ]:
## Display data and see if fused

# Note: If you chose to store all your data in one SQL Table,
# Dask DataFrames can import from that.
gdfbag = dfbag.map(gpd.GeoDataFrame.from_features)

In our case, our data is already fused. But often you will have several datasets with asynchronous data that you will have to fuse first. We implemented a barebones method in mm_utils to handle this; here is an example of how to apply it.

Note that your data needs to be a (Geo)DataFrame or GeoJSON. Also, the first column of all the datasets needs to be the time, and must all share the same time formatting. If you aren't sure your time format will work, we recommmend converting it all to Unix time (most languages have a built-in method to do this)

In [ ]:
# We create simulated asynchronous data

# Get columns of data
origdf = gdfbag.take(1)[0]

/home/gjgress/.local/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
speed = origdf[['timestamp','speed']]

# Create rng to create noisy data
rng = np.random.default_rng()
unfused_spd = pd.DataFrame([])
for i in speed.index[:-1]:
    # Generate speed values based on original, at random timestamps
    row1 = pd.DataFrame([[speed.iloc[i][0] + rng.random()*1000,(speed.iloc[i][1] + speed.iloc[i+1][1])/2 + np.random.normal(0, (np.abs(speed.iloc[i][1] - speed.iloc[i+1][1]))/4)]], columns = ['timestamp','speed'], index = [i])
    unfused_spd = pd.concat([unfused_spd, row])
# Reset indices
unfused_spd = unfused_spd.sort_index().reset_index(drop=True)
unfused_spd

,timestamp,speed
0,1.503834e+12,4.695797
1,1.503834e+12,4.479840
2,1.503834e+12,4.365951
3,1.503834e+12,4.174768
4,1.503834e+12,3.970857
5,1.503834e+12,3.870786
6,1.503834e+12,3.882143
7,1.503834e+12,4.311895
8,1.503834e+12,4.999623
9,1.503834e+12,5.771902


Now we wish to fuse this DataFrame with our original dataframe (excluding the original speed column)

In [ ]:
df1 = origdf[['timestamp', 'altitude', 'vertical accuracy', 'horizontal accuracy', 'geometry']]

df_prox = mm_utils.fuse(df1,unfused_spd,'timestamp','nearest neighbor')
df_avg = mm_utils.fuse(df1,unfused_spd,'timestamp','average')

Now let's see what the speed columns look like side-by-side

In [ ]:
# This cell sets up styling to facilitate comparison
from IPython.display import display_html

df_sty = origdf[['speed']].style.set_table_attributes("style='display:inline'").set_caption('original df')
df_prox_sty = df_prox[['speed']].style.set_table_attributes("style='display:inline'").set_caption('proximity fuse')
df_avg_sty = df_avg[['speed']].style.set_table_attributes("style='display:inline'").set_caption('average fuse')

In [ ]:
space = "\xa0" * 10
display_html(df_sty._repr_html_() + space
             + df_prox_sty._repr_html_() + space
             + df_avg_sty._repr_html_(), raw=True)


,speed
0,5.670000
1,4.610000
2,4.390000
3,4.350000
4,4.100000
5,3.890000
6,3.860000
7,3.890000
8,4.500000
9,5.610000


Because these are undersampled data points with heavy noise, the averaging method performs rather poorly. If sample size is far larger than the main dataset, average performs a lot better-- but then nearest neighbor is weak to high variance (as it only considers the closest point, which may be an outlier.

## Large-Scale Dataset

First, we import the data and write it as GEOJson

In [ ]:
# Processing the coordinate track first

import csv
from shapely.geometry import Point
from shapely.geometry import LineString
path = 'map-matching-dataset/'


In [ ]:

for filename in os.listdir(path):
    if '.track' in filename:
        gdf = gpd.GeoDataFrame([])
        
        filepath = os.path.join(path, filename)
        with open(filepath,newline='') as csvf:
            data = csv.reader(csvf,delimiter = '\t')
            # To be honest this is a lazy approach, but we only have 100 tracks anyway, so its fine...
            for row in data:
                pt = Point(float(row[0]), float(row[1]))
                gdf = pd.concat([gdf, gpd.GeoDataFrame([{'timestamp':row[2], 'geometry':pt}])])
        gdf.to_file(filepath[:-6] + 'track' + '.geojson', driver = 'GeoJSON')

In [ ]:
# Now we construct our network

def string_to_coords(string):
    return tuple(map(float,string.split('\t')))

def string_to_ints(string):
    return tuple(map(int,string.split('\t')))


def helper_fn1(df,ls):
    return df.iloc[ls]

def helper_fn2(ls):
    return LineString(list(ls.loc[:,'geometry']))

for filename in os.listdir(path):
    if '.nodes' in filename:
        filepath = os.path.join(path, filename)
        with open(filepath,newline='') as f:
            fdata = f.read()
            flist = fdata.split('\n')
            flist = list(filter(None, flist))
            flist = map(Point,list(map(string_to_coords,flist)))
            ngdf = gpd.GeoDataFrame(flist, columns=['geometry'])
            # We need to do arcs while we do nodes, b.c. it is indexed by rows in .nodes
            with open(filepath[:-6] + '.arcs', newline='') as f2:
                arcdata = f2.read()
                arclist = arcdata.split('\n')
                arclist = list(filter(None,arclist))
                arclist = list(map(string_to_ints,arclist))
                arclistu = [] # We need to store the source and target nodes
                arclistv = []
                for i in range(len(arclist)):
                    arclistu.append(arclist[i][0])
                    arclistv.append(arclist[i][1])
                arclist = map(list, arclist)
                arcpts = list(map(lambda x: helper_fn1(ngdf,x), arclist))
                arcptsid = list(map(lambda x: helper_fn1(ngdf,x), arclist))
                arclist = list(map(helper_fn2, arcpts))
                agdf = gpd.GeoDataFrame({'geometry' : arclist, 'u' : arclistu, 'v' : arclistv})
        ngdf.to_file(filepath[:-6] + 'nodes' + '.geojson', driver = 'GeoJSON')
        agdf.to_file(filepath[:-6] + 'arcs' + '.geojson', driver = 'GeoJSON')

In [ ]:
# Ground Truth
# This is the RIGHT way to do this... much faster. When I get time, rewrite the previous code block
for filename in os.listdir(path):
    if 'arcs.geojson' in filename:
        filepath = os.path.join(path, filename)
        agdf = gpd.read_file(filepath)
        with open(filepath[:-12] + '.route','r') as f:
            data = f.read()
            dlist = data.split("\n")
            dlist = list(filter(None, dlist))
            dlist = list(map(int,dlist))
            rgdf = agdf.iloc[dlist]
        rgdf.to_file(filepath[:-12] + 'route' + '.geojson', driver = 'GeoJSON', index = True)


In [ ]:
for filename in os.listdir(path):
    if ('geojson' in filename):
        filepath = os.path.join(path, filename)
        with open(filepath, 'r+') as f:
            text = f.read().replace('\n', '')
            f.seek(0)
            f.write(text)
            f.truncate()

0

300442

300442

0

1199263

...

586345

586345

0

6677294

6677294